In [1]:
import pandas as pd

In [4]:
row_number = df.index.get_loc(df[(df['orthopedic_surgeon_surname'] == 'Gross') & (df['orthopedic_surgeon_first_name'].str.contains('Thomas'))].index[0])

print(row_number)

530


In [2]:
from openai import OpenAI

client = OpenAI()

In [3]:
df = pd.read_csv('./Find-a-Doctor/data/hip_surgeons.csv')
documents = df.to_dict(orient='records')

In [4]:
prompt_template = """
You emulate a user of our find a hip surgeon application.
Formulate 5 questions this user might ask based on a provided doctor.
Make the questions specific to this doctor.
The record should contain the answer to the questions, and the questions should
be complete and not too short. Use as fewer words as possible from the record. 

The record:


orthopedic_surgeon_first_name: {orthopedic_surgeon_first_name}
orthopedic_surgeon_surname: {orthopedic_surgeon_surname}
jpg_doctor: {jpg_doctor}
numbers_hr: {number_hr}
adjusted: {adjusted}
number_thr: {number_thr}
number_bmhr: {number_bmhr}
type_of_hr_prosthesis: {type_of_hr_prosthesis}
type_of_thr_prosthesis: {type_of_thr_prosthesis}
operational_technique: {operational_technique}
anesthetic: {anesthetic}
cement_femur_side: {cement_femur_side}
this_joint_capsule_saved_all: {this_joint_capsule_saved_all}
city: {city}
country: {country}
patient-reported_positive_outcomes: {patient-reported_positive_outcomes}
patient-reported_complications: {patient-reported_complications}
cut_muscles_is_fixed_again: {cut_muscles_is_fixed_again}
complete_opera_report_is_given: {complete_opera_report_is_given}
hr_average_size: {hr_average_size}
hr_average_location: {hr_average_location}
assesses_x-ray: {assesses_x-ray}
x-ray_at_discharge: {x-ray_at_discharge}
two-sided_operation: {two-sided_operation}
foreign_patients: {foreign_patients}
address: {address}
street: {street}
local: {local}
postal_code: {postal_code}
phone_normal: {phone_normal}
phone_free_of_charge: {phone_free_of_charge}
fax: {fax}
mobile: {mobile}
email: {email}
homepage:{homepage}


Provide the output in parsable JSON without using code blocks:

{{"questions": ["question1", "question2", ..., "question5"]}}
""".strip()




In [5]:
prompt = prompt_template.format(**documents[0])

In [6]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [7]:
questions = llm(prompt)

In [8]:
import json

In [9]:
json.loads(questions)

{'questions': ["What is Dr. Michael Anderson's specialty and experience in hip surgeries?",
  "Can you provide the contact information for Dr. Michael Anderson's office in Murdoch?",
  'What types of prostheses does Dr. Anderson typically use for hip replacement surgery?',
  'Are there any patient-reported outcomes or complications from surgeries performed by Dr. Anderson?',
  'What is the address of the Murdoch University Medical Centre where Dr. Anderson practices?']}

In [10]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [11]:
from tqdm.auto import tqdm

In [12]:
results = {}

In [13]:
for index, doc in enumerate(tqdm(documents)): 
    doc_id = index#doc['id']
    if doc_id in results:
        continue

    questions_raw = generate_questions(doc)
    questions = json.loads(questions_raw)
    results[doc_id] = questions['questions']

  0%|          | 0/1032 [00:00<?, ?it/s]

In [14]:
final_results = []

for doc_id, questions in results.items():
    for q in questions:
        final_results.append((doc_id, q))

In [15]:
final_results[0]

(0, "What is Dr. Michael Anderson's experience with hip surgeries?")

In [16]:
df_results = pd.DataFrame(final_results, columns=['id', 'question'])

In [17]:
df_results.to_csv('./Find-a-Doctor/data/ground-truth-retrieval.csv', index=False)

In [18]:
!head ./Find-a-Doctor/data/ground-truth-retrieval.csv

id,question
0,What is Dr. Michael Anderson's experience with hip surgeries?
0,What types of hip replacement prostheses does Dr. Anderson use?
0,Are there any reported complications from patients who have undergone surgery with Dr. Anderson?
0,What is the location of Dr. Michael Anderson's practice?
0,How can I contact Dr. Anderson's office for more information?
1,What is Dr. Neil Bergman's experience in performing hip surgeries?
1,Can you provide details about the types of prostheses used by Dr. Bergman during hip replacement surgeries?
1,Is there a record of any patient-reported complications from surgeries conducted by Dr. Bergman?
1,What kind of anesthesia does Dr. Bergman typically use for hip surgeries?
